In [1]:
from dariah_topics import preprocessing as pre
from dariah_topics import visualization as visual
from dariah_topics import evaluation

c:\users\philip\appdata\local\programs\python\python35-32\lib\site-packages\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
c:\users\philip\appdata\local\programs\python\python35-32\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
c:\users\philip\appdata\local\programs\python\python35-32\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
c:\users\philip\appdata\local\programs\python\python35-32\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
c:\users\philip\appdata\local\programs\python\python35-32\lib\site-packages\funcy\decorators.py

## Preprocessing

#### Liste mit Dateinamen erzeugen

In [2]:
path_txt = "corpus_txt"
#path_txt = "grenzbote_plain/*/"
#path_txt = "wiki/"

doclist_txt = pre.create_document_list(path_txt)
assert doclist_txt, "No documents found"
doclist_txt[:5]

c:\users\philip\appdata\local\programs\python\python35-32\lib\site-packages\IPython\core\formatters.py:92: DeprecationWarning: DisplayFormatter._ipython_display_formatter_default is deprecated: use @default decorator instead.
  def _ipython_display_formatter_default(self):
c:\users\philip\appdata\local\programs\python\python35-32\lib\site-packages\IPython\core\formatters.py:98: DeprecationWarning: DisplayFormatter._formatters_default is deprecated: use @default decorator instead.
  def _formatters_default(self):
c:\users\philip\appdata\local\programs\python\python35-32\lib\site-packages\IPython\core\formatters.py:677: DeprecationWarning: PlainTextFormatter._deferred_printers_default is deprecated: use @default decorator instead.
  def _deferred_printers_default(self):
c:\users\philip\appdata\local\programs\python\python35-32\lib\site-packages\IPython\core\formatters.py:669: DeprecationWarning: PlainTextFormatter._singleton_printers_default is deprecated: use @default decorator instead.

['corpus_txt\\Doyle_AScandalinBohemia.txt',
 'corpus_txt\\Doyle_AStudyinScarlet.txt',
 'corpus_txt\\Doyle_TheHoundoftheBaskervilles.txt',
 'corpus_txt\\Doyle_TheSignoftheFour.txt',
 'corpus_txt\\Howard_GodsoftheNorth.txt']

#####  Liste mit Dokumentenlabels erzeugen - (Funktion wird durch Thorsten's generischere Funktion ersetzt)

In [3]:
doc_labels = list(pre.get_labels(doclist_txt))
doc_labels[:5]

['corpus_txt\\Doyle_AScandalinBohemia.txt',
 'corpus_txt\\Doyle_AStudyinScarlet.txt',
 'corpus_txt\\Doyle_TheHoundoftheBaskervilles.txt',
 'corpus_txt\\Doyle_TheSignoftheFour.txt',
 'corpus_txt\\Howard_GodsoftheNorth.txt']

#### Corpus laden

In [4]:
corpus_txt = pre.read_from_txt(doclist_txt)

#### Tokenisieren

In [5]:
doc_tokens = [list(pre.tokenize(txt)) for txt in list(corpus_txt)]

#### Create Dictionaries

In [6]:
id_types, doc_ids = pre.create_dictionaries(doc_labels, doc_tokens)

#### Sparse BOW Model

In [7]:
sparse_bow = pre.create_mm(doc_labels, doc_tokens, id_types, doc_ids)

In [8]:
sparse_bow[:5]

0
doc_id token_id    
1      16387      1
       20484     11
       12294      1
       20487      1
       20488      1

## Topic Modeling with Gensim

##### Saving Sparse BOW

In [9]:
pre.save_bow_mm(sparse_bow, "gensim_txt")

#### Import Market Matrix

In [10]:
from gensim.models import LdaModel
from gensim.corpora import MmCorpus

In [11]:
mm = MmCorpus("gensim_txt.mm")

In [12]:
doc2id = {value : key for key, value in doc_ids.items()}
type2id = {value : key for key, value in id_types.items()}

##### Remove Features

type2id = {value : key for key, value in id_types.items()}
sparse_bow_collapsed = sparse_bow.groupby(sparse_bow.index.get_level_values('token_id')).sum()

sparse_bow_hapax = sparse_bow_collapsed.loc[sparse_bow_collapsed[0] == 1]
hapax = [type2id[key] for key in sparse_bow_hapax.index.get_level_values('token_id')]

len(hapax)

In [13]:
import os.path
basepath = os.path.abspath('.')

with open(os.path.join(basepath, "tutorial_supplementals", "stopwords", "de.txt"), 'r', encoding = 'utf-8') as f: 
    stopword_list = f.read().split('\n')
    
stopword_list = set(stopword_list)

In [14]:
hapax_from_remove = pre.find_hapax(sparse_bow, id_types)
stopwords_from_remove = pre.find_stopwords(sparse_bow, id_types, mfw=75)

#features_to_be_removed = set(hapax_from_remove + stopwords_from_remove)
features_to_be_removed = stopwords_from_remove

sparse_bow_short = pre.remove_features(sparse_bow, id_types, features_to_be_removed)

In [15]:
len(features_to_be_removed)

75

In [16]:
pre.save_bow_mm(sparse_bow_short, "gensim_txt")

In [17]:
mm = MmCorpus("gensim_txt.mm")

#### Convert sparse_bow to list of (doc, tokens) tuples (like doc2bow)

In [18]:
doc2bow_list = []

for doc in sparse_bow_short.index.groupby(sparse_bow_short.index.get_level_values('doc_id')):
    temp = [(token, count) for token, count in zip(sparse_bow_short.loc[doc].index, sparse_bow_short.loc[doc][0])]
    doc2bow_list.append(temp)

In [19]:
len(doc2bow_list)

16

In [20]:
#model = LdaModel(corpus=mm, id2word=type2id, num_topics=60, alpha = "symmetric", passes = 10) #import momentan in visual 
# -> da ich mir noch nicht sicher bin, welche Funktionen in das tm_gensim.py sollen
model = LdaModel(corpus=mm, id2word=type2id, num_topics=20, passes = 10, iterations = 20)

In [21]:
model.get_document_topics(doc2bow_list[1])

[(0, 0.012883995158465566),
 (4, 0.19288311761917948),
 (5, 0.04973348232002954),
 (6, 0.016222277513200763),
 (8, 0.02331851033351949),
 (9, 0.36808773472395012),
 (12, 0.11374919978057629),
 (14, 0.11883524242343806),
 (19, 0.089757021026767886)]

In [22]:
#Anzeige der keywords für topic n
n = 1
topic_nr_x = model.get_topic_terms(n)

topicTerms = [type2id[i[0]] for i in topic_nr_x]
topicTerms

['assured',
 'gnashing',
 'pranks',
 'confession',
 'pea-shooter',
 'sentry',
 'new-york',
 'termed',
 'apples',
 'hansom']

In [23]:
#Topic probabilities
topics = model.show_topics(num_topics = 40)
topics

[(0,
  '0.005*"menace" + 0.004*"hornets" + 0.004*"littlest" + 0.003*"termed" + 0.003*"apples" + 0.003*"implore" + 0.003*"roofless" + 0.003*"introspective" + 0.003*"coincide" + 0.003*"nervous"'),
 (1,
  '0.000*"assured" + 0.000*"gnashing" + 0.000*"pranks" + 0.000*"confession" + 0.000*"pea-shooter" + 0.000*"sentry" + 0.000*"new-york" + 0.000*"termed" + 0.000*"apples" + 0.000*"hansom"'),
 (2,
  '0.001*"inaction" + 0.001*"new-york" + 0.000*"confession" + 0.000*"lanky" + 0.000*"assured" + 0.000*"pea-shooter" + 0.000*"sentry" + 0.000*"cabdriver" + 0.000*"local" + 0.000*"termed"'),
 (3,
  '0.000*"assured" + 0.000*"collected" + 0.000*"confession" + 0.000*"pea-shooter" + 0.000*"bitten" + 0.000*"sentry" + 0.000*"stone-cast" + 0.000*"gulfs" + 0.000*"new-york" + 0.000*"with"'),
 (4,
  '0.004*"gnashing" + 0.004*"hornets" + 0.003*"sentry" + 0.003*"stone-cast" + 0.003*"local" + 0.003*"assured" + 0.003*"separateness" + 0.003*"excesses" + 0.003*"with" + 0.003*"luminous"'),
 (5,
  '0.016*"drop" + 0.005*

In [39]:
import pandas as pd
import regex as re
def gensim2dataframe(model):
    num_topics = model.num_topics
    topics_df = pd.DataFrame(index = range(num_topics), columns= range(10))

    topics = model.show_topics(num_topics)
    
    for topic_dist in topics:    
        idx = topic_dist[0]
        temp = re.findall(r'\"(.+?)\"', topics[idx][1])
        topics_df.loc[idx] = temp
    
    return topics_df

In [40]:
topics_df = gensim2dataframe(model)

In [41]:
topics_df

,0,1,2,3,4,5,6,7,8,9
0,menace,hornets,littlest,termed,apples,implore,roofless,introspective,coincide,nervous
1,assured,gnashing,pranks,confession,pea-shooter,sentry,new-york,termed,apples,hansom
2,inaction,new-york,confession,lanky,assured,pea-shooter,sentry,cabdriver,local,termed
3,assured,collected,confession,pea-shooter,bitten,sentry,stone-cast,gulfs,new-york,with
4,gnashing,hornets,sentry,stone-cast,local,assured,separateness,excesses,with,luminous
5,drop,expressly,serious,hornets,henrietta,authentically,momentum,pranks,excesses,termed
6,peeling,chuckle,internal,lot,nonchalance,veil,invalid,bruises,perversity,need
7,dumps,linkages,hornets,devil,aesthetic,expressly,objective,gnashing,puffs,henrietta
8,yelped,rolled,splashing,dolphin-jump,terribly,plenty-more,admired,miskatonic,grievances,termed
9,inaction,sentry,confession,assured,collected,momentum,terribly,lanky,bitten,hansom


## Visualisierung 

In [ ]:
'''lda_model = 'out_easy/corpus.lda'
corpus = 'out_easy/corpus.mm'
dictionary = 'out_easy/corpus.dict'
doc_labels = 'out_easy/corpus_doclabels.txt'
interactive  = False

vis = visual.Visualization(lda_model, corpus, dictionary, doc_labels, interactive)'''

##### Create a document-topic matrix (that is a pandas data frame actually)

In [ ]:
doc_topic = visual.create_doc_topic(mm, model, doc_labels)

##### Visualize document-topic matrix

In [ ]:
%matplotlib inline
visual.doc_topic_heatmap(doc_topic)

##### Visualize topic distribution in a single document

In [ ]:
visual.plot_doc_topics(doc_topic, 0)

![success](http://cdn2.hubspot.net/hub/128506/file-446943132-jpg/images/computer_woman_success.jpg)